In [1]:
import sys, os, git

current_path = os.getcwd()
git_repo = git.Repo(current_path, search_parent_directories=True)
git_path = git_repo.git.rev_parse("--show-toplevel")
sys.path.append(git_path+"/python-codes/")

from InitializeSimulation import InitializeSimulation
from Utilities import Utilities
from Outputs import Outputs
from MolecularDynamics import MolecularDynamics
from MonteCarlo import MonteCarlo

import numpy as np

In [ ]:
self = MolecularDynamics(number_atoms=[50, 10],
                      epsilon=[0.1, 0.1], # kcal/mol
                      sigma=[2, 1], # A
                      atom_mass= [1, 1], # g/mol
                      Lx=30, # A
                      Ly=30, # A
                      Lz=30, # A
                      minimization_steps = 20,
                      maximum_steps=5000,
                      desired_temperature=300,
                      thermo = 10,
                      dump = 10,  
                      tau_temp = 100, # fs
                      #tau_press= 1000, # fs
                      time_step=1, # fs
                      seed=319817,
                      data_folder = "mdcode-output/",
                      )
self.run()
# run lammps for comparison
os.system("/home/simon/Softwares/lammps-2Aug2023/src/lmp_serial -in input.lammps")

step epot maxF
0 662.005 211481.127
5 27.189 6435.010
10 -0.963 8.846
15 -1.211 4.505
20 -1.283 7.241
 step     N  temp  epot  ekin press   vol
    0    60 299.97 -1.2778678594397708 52.7546809709511 87.95 27000.0
   10    60 297.52 -0.82495404799328 52.32462368756891  93.0 27000.0
   20    60 298.75 -1.008156288511023 52.53981019506951 89.36 27000.0
   30    60 298.85 -1.005778684429036 52.558697201505645 87.25 27000.0
   40    60 298.61 -0.9439274249040279 52.515721363475365 87.23 27000.0
   50    60 294.25 -0.0947898243971407 51.74920793250246 95.59 27000.0
LAMMPS (2 Aug 2023 - Update 1)
Reading data file ...
  orthogonal box = (-15 -15 -15) to (15 15 15)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  60 atoms
  reading velocities ...
  60 velocities
  read_data CPU = 0.001 seconds
Generated 1 of 1 mixed pair_coeff terms from geometric mixing rule
Neighbor list info ...
  update: every = 1 steps, delay = 0 steps, check = yes
  max neighbors/atom: 2000, page size: 100000
  ma

0

In [3]:
self.neighbor_lists

[[1, 13, 45, 46, 47, 48, 54, 58],
 [0, 11, 13, 19, 24, 34, 36, 38, 45, 48, 52, 54, 59],
 [7, 16, 23, 33, 37, 39, 42, 51],
 [6, 7, 10, 21, 32, 43, 53, 55, 57],
 [8, 20, 22, 27, 31, 40, 41, 49, 50, 55],
 [8, 12, 19, 31, 39, 41, 49],
 [3, 10, 17, 21, 24, 32, 33, 43, 48, 53, 57],
 [2, 3, 9, 16, 23, 37, 39, 42, 51],
 [4, 5, 16, 27, 31, 33, 34, 37, 40, 41, 44, 48, 49, 50, 57],
 [7, 23, 30, 42, 51, 53, 56],
 [3, 6, 11, 21, 33, 36, 38, 46, 53, 59],
 [1, 10, 19, 33, 36, 37, 54, 59],
 [5, 14, 18, 41, 56],
 [0, 1, 38, 48, 52, 54, 56, 58, 59],
 [12, 18, 25, 28, 29],
 [21, 24, 26, 52],
 [2, 7, 8, 31, 33, 37, 39, 41, 42, 49, 51, 57],
 [6, 32, 34, 43, 44, 52, 57],
 [12, 14, 25, 28, 29, 55],
 [1, 5, 11, 54],
 [4, 22, 28, 45],
 [3, 6, 10, 15, 24, 26, 43, 53],
 [4, 20, 27, 31, 40, 41, 49, 50, 55],
 [2, 7, 9, 30, 37, 39, 42, 51, 53],
 [1, 6, 15, 21, 34, 52, 57],
 [14, 18, 29, 55, 56],
 [15, 21, 38, 52],
 [4, 8, 22, 29, 31, 40, 41, 44, 46, 47, 48, 49, 50],
 [14, 18, 20, 29, 45],
 [14, 18, 25, 27, 28, 40, 

In [4]:
import random
import time

positions = []
cut_off_distance=2
num_particles = 100
total_iterations = 10000

#Create a list of random particle positions
for i in range(num_particles):
  
  while True:
    rand_pos = round(random.uniform(0, 25), 3) #make sure not to add them in the same place

    if not(rand_pos in positions):  
      positions.append(rand_pos) #pick a random position between 0-25
      break
      
##################################
## Aproach 1 - no neighbor list
##################################

#Begin "brute force" calculation of the forces
brute_start = time.perf_counter()

for iter in range(total_iterations):

  for i in range(num_particles): #loop through all particles
    for j in range(num_particles): #loop through the rest of the particles
      if i==j:
        continue #skip any self-forces

      if abs(positions[i]-positions[j])<cut_off_distance:
        F = 10/(positions[i]-positions[j]) #arbitrary stand-in of force calculation

brute_end = time.perf_counter()

##################################
## Aproach 2 - with neighbor list
##################################

neighbor_list = []

#Begin calculation of the forces using a neighbor list
neighbor_start = time.perf_counter()

for iter in range(total_iterations):

  #Update the neighbor list every 10 iterations
  if iter%10==0:
    neighbor_list = []
    for i in range(num_particles): #loop through all particles

      a_list = []

      for j in range(num_particles): #loop through the rest of the particles
        if i==j:
          continue #skip any self-forces

        if abs(positions[i]-positions[j])<cut_off_distance:
          a_list.append(j)

      neighbor_list.append(a_list.copy())

  for i in range(num_particles): #loop through all particles
    for neighbor in neighbor_list[i]: #go through each particle in its neighbor list
      F = 10/abs(positions[i]-positions[neighbor]) #arbitrary stand-in of force calculation

neighbor_end = time.perf_counter()


print("No neighbor list: " + str(round(brute_end-brute_start,2)) + "s")
print("Using a neighbor list: " + str(round(neighbor_end-neighbor_start,2)) + "s\n")

#Print some of the neighbor lists
for i in range(4):
  print("Particle " + str(i) + " neighbors: " + str(neighbor_list[i]))